In [58]:
#https://analyticsindiamag.com/guide-to-building-and-deploying-ml-web-applications-using-pycaret-streamlit-and-heroku/
!pip install pycaret


     |████████████████████████████████| 256kB 2.8MB/s 
     |████████████████████████████████| 65.9MB 99kB/s 
     |████████████████████████████████| 102kB 12.9MB/s 
     |████████████████████████████████| 12.4MB 48.3MB/s 
     |████████████████████████████████| 266kB 49.7MB/s 
     |████████████████████████████████| 6.8MB 47.5MB/s 
     |████████████████████████████████| 266kB 49.3MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 1.2MB 40.4MB/s 
     |████████████████████████████████| 1.6MB 42.0MB/s 
     |████████████████████████████████| 2.1MB 43.8MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 327kB 46.4MB/s 
     |████████████████████████████████| 163kB 52.0MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
     |████████████████████████████████| 153kB 55.9MB/s 
     |████████████████████████████████| 6.0MB 47.9MB/s 
     |████████████████████████████████| 1.1MB 42.7MB/s

In [1]:
#Once the installation is done, we will load the dataset as follows. 
from pycaret.datasets import get_data
dataset = get_data('employee')

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,department,salary,left
0,0.38,0.53,2,157,3,0,0,sales,low,1
1,0.80,0.86,5,262,6,0,0,sales,medium,1
2,0.11,0.88,7,272,4,0,0,sales,medium,1
3,0.72,0.87,5,223,5,0,0,sales,low,1
4,0.37,0.52,2,159,3,0,0,sales,low,1


In [3]:
data_seen = dataset.sample(frac=0.96, random_state=780).reset_index(drop=True)
data_unseen = dataset.drop(data_seen.index).reset_index(drop=True)

dataset=dataset.drop(['department','average_montly_hours'],axis=1)
print('Data for Modelling '+str(data_seen.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modelling (14399, 10)
Unseen Data For Predictions: (600, 10)


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   time_spend_company     14999 non-null  int64  
 4   Work_accident          14999 non-null  int64  
 5   promotion_last_5years  14999 non-null  int64  
 6   salary                 14999 non-null  object 
 7   left                   14999 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 937.6+ KB


In [5]:
from pycaret.classification import *
setting_up = setup(data = data_seen, target = 'left', session_id=123, silent=True, sampling=False)

Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(14399, 10)"
4,Missing Values,False
5,Numeric Features,3
6,Categorical Features,6
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [7]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Random Forest Classifier,0.9855,0.9869,0.9512,0.9875,0.9689,0.9595,0.9598,0.1110
1,Light Gradient Boosting Machine,0.9853,0.9918,0.9508,0.9870,0.9686,0.9590,0.9593,0.1732
2,CatBoost Classifier,0.9825,0.9914,0.9416,0.9843,0.9624,0.9511,0.9515,4.7255
3,Extra Trees Classifier,0.9814,0.9884,0.9562,0.9655,0.9608,0.9487,0.9487,0.5480
4,Gradient Boosting Classifier,0.9758,0.9859,0.9229,0.9741,0.9477,0.9320,0.9326,1.1392
5,Extreme Gradient Boosting,0.9749,0.9853,0.9179,0.9752,0.9456,0.9293,0.9301,0.6867
6,Decision Tree Classifier,0.9739,0.9698,0.9621,0.9308,0.9461,0.9289,0.9292,0.0442
7,Ada Boost Classifier,0.9580,0.9805,0.9033,0.9196,0.9111,0.8837,0.8839,0.4029
8,K Neighbors Classifier,0.9315,0.9685,0.8966,0.8306,0.8620,0.8166,0.8179,0.0409
9,Logistic Regression,0.8902,0.9325,0.7778,0.7653,0.7710,0.6988,0.6992,0.2146


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=123, verbose=0,
                       warm_start=False)

In [8]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9881,0.9928,0.9542,0.9957,0.9745,0.9667,0.9671
1,0.9881,0.9875,0.9625,0.9872,0.9747,0.9669,0.9670
2,0.9921,0.9990,0.9750,0.9915,0.9832,0.9780,0.9781
3,0.9831,0.9859,0.9458,0.9827,0.9639,0.9529,0.9532
4,0.9921,0.9885,0.9708,0.9957,0.9831,0.9779,0.9781
5,0.9841,0.9826,0.9458,0.9870,0.9660,0.9556,0.9560
6,0.9841,0.9814,0.9500,0.9828,0.9661,0.9557,0.9560
7,0.9831,0.9781,0.9542,0.9745,0.9642,0.9532,0.9533
8,0.9831,0.9924,0.9333,0.9956,0.9634,0.9525,0.9533
9,0.9772,0.9805,0.9205,0.9821,0.9503,0.9355,0.9363


In [9]:
tuned_model = tune_model(rf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9861,0.9897,0.9458,0.9956,0.9701,0.9610,0.9616
1,0.9841,0.9912,0.9458,0.9870,0.9660,0.9556,0.9560
2,0.9921,0.9960,0.9667,1.0000,0.9831,0.9779,0.9781
3,0.9812,0.9860,0.9250,0.9955,0.9590,0.9468,0.9478
4,0.9861,0.9868,0.9458,0.9956,0.9701,0.9610,0.9616
5,0.9782,0.9917,0.9125,0.9955,0.9522,0.9381,0.9396
6,0.9861,0.9825,0.9500,0.9913,0.9702,0.9612,0.9615
7,0.9831,0.9843,0.9458,0.9827,0.9639,0.9529,0.9532
8,0.9812,0.9942,0.9250,0.9955,0.9590,0.9468,0.9478
9,0.9752,0.9804,0.9079,0.9864,0.9455,0.9295,0.9308


In [10]:
final = finalize_model(tuned_model)
unseen_predictions = predict_model(final, data=data_unseen)
unseen_predictions.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,department,salary,left,Label,Score
0,0.37,0.55,2,152,3,0,0,technical,medium,1,1,0.9833
1,0.44,0.51,2,156,3,0,0,technical,high,1,1,1.0000
2,0.09,0.80,7,283,5,0,0,technical,low,1,1,1.0000
3,0.92,0.87,4,226,6,1,0,technical,medium,1,1,0.7405
4,0.74,0.91,4,232,5,0,0,technical,medium,1,1,1.0000


In [11]:
save_model(final,'Final_model')

Transformation Pipeline and Model Succesfully Saved


In [13]:
#!pip install streamlit

     |████████████████████████████████| 7.2MB 2.8MB/s 
     |████████████████████████████████| 102kB 12.9MB/s 
     |████████████████████████████████| 4.4MB 43.4MB/s 
     |████████████████████████████████| 112kB 26.6MB/s 
     |████████████████████████████████| 122kB 47.7MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73870 sha256=344f62932d6c9cc80727a6c7534bce7ea3794f0de7dd7f9e0427a4b479b2db5e
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13449 sha256=c4fd14e58bea7a48455c54ecafdd791f42708a15ec3c3942a5ce78c6de5087a6
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8784 sha256=f06a468eb00ea13ac602f411bfa616f2218fd446b517b66d9e72f0afe6aaba82
  Stored in directory: /root/.cache/pip/whee

In [16]:
from pycaret.classification import load_model, predict_model
import streamlit as st
import pandas as pd
import numpy as np
model = load_model('Final_model')

Transformation Pipeline and Model Sucessfully Loaded


In [17]:
def predict(model, input_df):
    predictions_df = predict_model(estimator=model, data=input_df)
    predictions = predictions_df['Label'][0]
    return predictions

In [23]:
def run():
    from PIL import Image
    image = Image.open('Employee.png')
    image_hospital = Image.open('office.jpg')
    st.image(image,use_column_width=False)
    add_selectbox = st.sidebar.selectbox(
    "How would you like to predict?",
    ("Online", "Batch"))
    st.sidebar.info('This app is created to predict if an employee will leave the company')
    st.sidebar.success('https://www.pycaret.org')
    st.sidebar.image(image_hospital)
    st.title("Predicting employee leaving")
    if add_selectbox == 'Online':
        satisfaction_level=st.number_input('satisfaction_level' , min_value=0.1, max_value=1.0, value=0.1)
        last_evaluation =st.number_input('last_evaluation',min_value=0.1, max_value=1.0, value=0.1)
        number_project = st.number_input('number_project', min_value=0, max_value=50, value=5)
        time_spend_company = st.number_input('time_spend_company', min_value=1, max_value=10, value=3)
        Work_accident = st.number_input('Work_accident',  min_value=0, max_value=50, value=0)
        promotion_last_5years = st.number_input('promotion_last_5years',  min_value=0, max_value=50, value=0)
        salary = st.selectbox('salary', ['low', 'high','medium'])
        output=""
        input_dict={'satisfaction_level':satisfaction_level,'last_evaluation':last_evaluation,'number_project':number_project,'time_spend_company':time_spend_company,'Work_accident': Work_accident,'promotion_last_5years':promotion_last_5years,'salary' : salary}
        input_df = pd.DataFrame([input_dict])
        if st.button("Predict"):
          output = predict(model=model, input_df=input_df)
          output = str(output)
          st.success('The output is {}'.format(output))
    if add_selectbox == 'Batch':
        file_upload = st.file_uploader("Upload csv file for predictions", type=["csv"])
        if file_upload is not None:
          data = pd.read_csv(file_upload)
          predictions = predict_model(estimator=model,data=data)
          st.write(predictions)



In [22]:
streamlit run "aim_deploying_ml_web_applications_using_pycaret_streamlit_and_heroku_.py"

SyntaxError: ignored